In [2]:
import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from preprocessing.data_prep import preprocess
from scipy.spatial import ConvexHull
from PIL import Image
import cairosvg
from xml.dom import minidom
from svg.path import parse_path, CubicBezier
import math
import numpy as np
import cv2
import random
import string

from statistics import mean, stdev 

# play around with these 
ALPHA = 0.75
BETA = 0.005
SPAWN_COUNT = 5

basefiles = preprocess('/Users/anjueappen/svg')
   
def compute_prob(string, index):
    order = index
    try:
        length = parse_path(string).length()
    except ZeroDivisionError:
        length = 0
    return math.exp(ALPHA*order)/math.exp(BETA*length)

def distort(string):
    parsed = parse_path(string)
    for i in range(len(parsed)):
        cb = parsed[i]
        if type(cb) == CubicBezier:
            std = np.abs(cb.start - cb.end)/cb.length()
            cb.control1 += np.random.normal(1, 2*std, 1)[0]
            cb.control2 += np.random.normal(1, 2*std, 1)[0]
    return parsed.d()
 
for basefile, _ in basefiles[:5]:
    doc = minidom.parse(basefile)
    path_strings = [(path, compute_prob(path.getAttribute('d'), index)) for index, path
                    in enumerate(doc.getElementsByTagName('path'))]
    
    vals = [item[1] for item in path_strings]
    avg = mean(vals)
    
    path_strings = [(s, prob/avg) for s, prob in path_strings]
    
    for i in range(SPAWN_COUNT):
        path_strings.remove(max(path_strings, key=lambda x: x[1])) 
        
        elements = []
        for s, _ in path_strings:
            distorted = distort(s.getAttribute('d'))
            if distorted is not None:
                s.setAttribute('d', distorted)
            elements.append(s)
            doc.childNodes[1].childNodes[1].childNodes[1].childNodes = elements
            
        tag = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(4))
        spawn_file = basefile.replace('svg', 'png').replace('.png', '_%s.png' % tag)
        svg_code = doc.toxml()
        
        with open(spawn_file,'w') as fout:
            cairosvg.svg2png(bytestring=svg_code, write_to=fout)
        
#         global_deformation(spawn_file)
        
    doc.unlink()


In [3]:
import numpy as np
import cv2
from scipy.spatial import ConvexHull
from PIL import Image 
from math import sqrt

def centeroid(data):
    x, y = zip(*data)
    l = len(x)
    return sum(x) / l, sum(y) / l

# open the image
img_array = Image.open('/Users/anjueappen/png/airplane/1_3.png')

# get coordinates of all black pixels
points = np.transpose(np.nonzero(np.array(img_array)))[:,:2]

# get points that comprise the convex hull 
hull_indicies = ConvexHull(points).vertices
hull_points = points[hull_indicies]

# calculate the centeroid of the convex hull 
centroid = centeroid(hull_points)

dist = lambda (x1, y1), (x2, y2): sqrt( (x2 - x1)**2 + (y2 - y1)**2 )

# introduce Gaussian noise to the image 
transformed_hull_points = [[int(round(point[0] + np.random.normal(1, 1/dist(point, centroid), 1)[0], 0)), 
                            int(round(point[1] + np.random.normal(1, 1/dist(point, centroid), 1)[0], 0))]
                            for point in hull_points]
print hull_points
print transformed_hull_points 

for pt_index in points:
    pass
    
c_original = centeroid(points)

IOError: [Errno 2] No such file or directory: '/Users/anjueappen/png/airplane/1_3.png'

In [ ]:
# def global_deformation(png_filepath):
#     img = cv2.imread(png_filepath)
#     ret,thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
#     im2,contours,hierarchy = cv2.findContours(thresh, 1, 2)
    
#     img_array = Image.open(png_filepath)
#     hull = ConvexHull(list(img_array.getdata()))
    
    
   